<a href="https://colab.research.google.com/github/Michila0/pose-detection/blob/main/VedioConvertImages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Upload the Video to Google Colab

In [ ]:
from google.colab import files
uploaded = files.upload()

# Install Required Libraries

In [ ]:
!pip install opencv-python

# Extract Frames from the Video

In [ ]:
import cv2
import os
from google.colab.patches import cv2_imshow

# Load the video
video_path = list(uploaded.keys())[0]  # Get the uploaded video file name
cap = cv2.VideoCapture(video_path)

# Create a folder to save extracted images
image_save_folder = "extracted_images"
if not os.path.exists(image_save_folder):
    os.makedirs(image_save_folder)

# Extract frames
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Save every nth frame
    if frame_count % 1 == 0:
        image_path = os.path.join(image_save_folder, f"frame_{frame_count}.jpg")
        cv2.imwrite(image_path, frame)
        print(f"Saved {image_path}")

    frame_count += 1

cap.release()
print(f"Extracted {frame_count} frames.")

#  View Extracted Images

In [ ]:
import matplotlib.pyplot as plt
import os

# List all extracted images
image_files = os.listdir(image_save_folder)

# Display the first few images
for i, image_file in enumerate(image_files[:5]):  # Display first 5 images
    image_path = os.path.join(image_save_folder, image_file)
    image = cv2.imread(image_path)
    plt.figure(figsize=(10, 5))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(image_file)
    plt.axis('off')
    plt.show()

# Track the Ball

In [ ]:
# Install the ultralytics package
!pip install ultralytics

In [ ]:
import cv2
from ultralytics import YOLO

# Load YOLOv8 model (pre-trained on a custom dataset for cricket ball detection)
model = YOLO("yolov8n.pt")  # Replace with a custom-trained model if available

# Variables to track ball state
ball_in_hand = False
ball_released = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection using YOLOv8
    results = model(frame)

    # Loop through detected objects
    for result in results:
        boxes = result.boxes.xyxy  # Bounding box coordinates
        confidences = result.boxes.conf  # Confidence scores
        class_ids = result.boxes.cls  # Class IDs

        for box, conf, cls_id in zip(boxes, confidences, class_ids):
            if model.names[int(cls_id)] == "cricket_ball":  # Check if the detected object is a cricket ball
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box

                # Check if the ball is in the bowler's hand (based on position)
                if y1 > 100:  # Adjust this condition based on your video
                    ball_in_hand = True
                    cv2.putText(frame, "Ball in Hand", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                else:
                    if ball_in_hand:  # Ball was in hand and now released
                        ball_released = True
                        cv2.putText(frame, "Ball Released", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # Display the frame
    cv2.imshow("Cricket Ball Tracking", frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Pose Estimation + Motion Analysis
This method uses pose estimation to track the batsman's body joints and analyzes their motion to detect batting actions.

## Install Libaries

In [ ]:
!pip install mediapipe opencv-python numpy matplotlib

In [ ]:
!pip uninstall mediapipe -y # uninstall existing mediapipe
!pip install mediapipe==0.10.20 # install specific version

## Detect Batting Pose with MediaPipe
Use MediaPipe to track the batsman's body joints(e.g: arms, shoulders, wrists) and analyze their movement over time.


In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import os

#Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# Process frames from the extracted images folder
image_save_folder = "extracted_images"
image_files = sorted(os.listdir(image_save_folder))

# Store joint angles or positions
batting_frames = []

for image_file in image_files:
    image_path = os.path.join(image_save_folder, image_file)
    frame = cv2.imread(image_path)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect pose
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        # Extract key joints (e.g., right wrist, left wrist, shoulders)
        landmarks = results.pose_landmarks.landmark
        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

        # Calculate bat swing angle (example logic)
        angle = np.arctan2(right_wrist[1] - left_wrist[1], right_wrist[0] - left_wrist[0])

        # Detect swing (threshold-based)
        if abs(angle) > 1.5:  # Adjust based on your video
            batting_frames.append(image_file)
            print(f"Batting action detected in {image_file}")

pose.close()

## Visualize Batting Action
Plot frames where batting is detected:

In [ ]:
for batting_frame in batting_frames[:10]:  # Show first 3 detected frames
    image_path = os.path.join(image_save_folder, batting_frame)
    image = cv2.imread(image_path)
    plt.figure(figsize=(10, 5))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Batting Action")
    plt.axis('off')
    plt.show()

#  Full Code for Batting Classification

In [ ]:
!pip install mediapipe opencv-python numpy matplotlib

In [ ]:
import cv2
import os
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# Define shot classification rules
def classify_shot(landmarks):
    # Extract key landmarks
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

    # Calculate bat angle (between wrists)
    bat_angle = np.arctan2(
        right_wrist.y - left_wrist.y,
        right_wrist.x - left_wrist.x
    )

    # Calculate shoulder-hip alignment (front/back foot)
    shoulder_hip_diff = abs(left_shoulder.y - right_hip.y)

    # Rule-based classification
    if bat_angle < -0.5:
        return "Cover Drive"
    elif bat_angle > 0.5:
        return "Pull Shot"
    elif shoulder_hip_diff < 0.1:
        return "Straight Drive"
    elif left_wrist.y < left_hip.y:
        return "Hook Shot"
    else:
        return "Defensive Shot"

# Process extracted images
image_save_folder = "extracted_images"  # Update with your folder path
# Check if the directory exists, and if not, create it
if not os.path.exists(image_save_folder):
    os.makedirs(image_save_folder)
    print(f"Directory '{image_save_folder}' created.")
else:
    print(f"Directory '{image_save_folder}' already exists.")

image_files = sorted(os.listdir(image_save_folder))

# Store results
batting_shots = []

for image_file in image_files:
    image_path = os.path.join(image_save_folder, image_file)
    frame = cv2.imread(image_path)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Pose detection
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        # Classify shot
        shot_name = classify_shot(results.pose_landmarks.landmark)
        batting_shots.append((image_file, shot_name))

        # Draw landmarks and shot name
        mp.solutions.drawing_utils.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
        )
        cv2.putText(frame, f"Shot: {shot_name}", (50, 50),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Save annotated frame
        output_path = os.path.join(image_save_folder, f"annotated_{image_file}")
        cv2.imwrite(output_path, frame)
    else:
        batting_shots.append((image_file, "No Pose Detected"))

pose.close()

# Display results
for (image_file, shot_name) in batting_shots[:5]:  # Show first 5 results
    image_path = os.path.join(image_save_folder, f"annotated_{image_file}")
    if os.path.exists(image_path):
        image = cv2.imread(image_path)
        plt.figure(figsize=(10, 6))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title(f"Shot: {shot_name}")
        plt.axis('off')
        plt.show()
    else:
        print(f"No pose detected in {image_file}")

In [ ]:
import cv2
import os
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.7)

# Define key landmarks
LANDMARKS = mp_pose.PoseLandmark

def classify_shot(landmarks):
    # Extract key joint positions
    left_wrist = [landmarks[LANDMARKS.LEFT_WRIST].x, landmarks[LANDMARKS.LEFT_WRIST].y]
    right_wrist = [landmarks[LANDMARKS.RIGHT_WRIST].x, landmarks[LANDMARKS.RIGHT_WRIST].y]
    left_shoulder = [landmarks[LANDMARKS.LEFT_SHOULDER].x, landmarks[LANDMARKS.LEFT_SHOULDER].y]
    right_shoulder = [landmarks[LANDMARKS.RIGHT_SHOULDER].x, landmarks[LANDMARKS.RIGHT_SHOULDER].y]
    left_hip = [landmarks[LANDMARKS.LEFT_HIP].x, landmarks[LANDMARKS.LEFT_HIP].y]
    right_hip = [landmarks[LANDMARKS.RIGHT_HIP].x, landmarks[LANDMARKS.RIGHT_HIP].y]

    # Calculate key metrics
    bat_angle = np.arctan2(right_wrist[1] - left_wrist[1], right_wrist[0] - left_wrist[0])
    shoulder_hip_alignment = abs(left_shoulder[1] - right_hip[1])
    wrist_hip_ratio = left_wrist[1] - left_hip[1]

    # Shot classification rules
    if bat_angle < -0.8:
        return "Cover Drive"
    elif bat_angle > 0.8:
        if wrist_hip_ratio < -0.1:
            return "Pull Shot"
        else:
            return "Hook Shot"
    elif shoulder_hip_alignment < 0.1:
        return "Straight Drive"
    elif abs(bat_angle) < 0.3 and wrist_hip_ratio < -0.15:
        return "Cut Shot"
    elif left_wrist[1] < left_hip[1] and right_wrist[1] < right_hip[1]:
        return "Sweep Shot"
    else:
        return "Defensive Shot"

# Process frames
image_folder = "/content/extracted_images"  # Path to your extracted frames
output_folder = "/content/annotated_images"
os.makedirs(output_folder, exist_ok=True)

shot_results = []

for img_file in sorted(os.listdir(image_folder)):
    img_path = os.path.join(image_folder, img_file)
    frame = cv2.imread(img_path)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        # Classify shot
        shot_name = classify_shot(results.pose_landmarks.landmark)

        # Draw annotations
        mp.solutions.drawing_utils.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp.solutions.drawing_styles.get_default_pose_landmarks_style()
        )

        # Add shot name and key lines
        cv2.putText(frame, f"Shot: {shot_name}", (20, 50),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Draw bat angle line
        h, w = frame.shape[:2]
        cv2.line(frame,
                (int(left_wrist[0]*w), int(left_wrist[1]*h)),
                (int(right_wrist[0]*w), int(right_wrist[1]*h)),
                (0, 0, 255), 3)

        # Save and store results
        output_path = os.path.join(output_folder, f"annotated_{img_file}")
        cv2.imwrite(output_path, frame)
        shot_results.append((img_file, shot_name))

    else:
        shot_results.append((img_file, "No Pose Detected"))

pose.close()

# Display results
for result in shot_results[:5]:  # Show first 5 results
    img_file, shot_name = result
    img_path = os.path.join(output_folder, f"annotated_{img_file}")

    if os.path.exists(img_path):
        img = cv2.imread(img_path)
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(f"Detected Shot: {shot_name}", fontsize=14)
        plt.axis('off')
        plt.show()
    else:
        print(f"Could not process: {img_file}")